In [24]:
# ====  這些是removebg 和clarifai 套件  ========
import requests
from clarifai.rest import Image as ClImage
from clarifai.rest import ClarifaiApp
# =============================================

# =========  這是模型檔的套件  ===============
from sklearn.ensemble import RandomForestClassifier
from sklearn.externals import joblib
# ============================================

import mysql.connector
import pandas as pd 
import numpy as np
import os
import math


In [25]:
clf = joblib.load('../models/scol_model.pkl')

In [42]:
def remove_bg(testimg):


    # 連線removebg
    response = requests.post(
        'https://api.remove.bg/v1.0/removebg',
        files={'image_file': open(testimg, 'rb')},
        data={'size': 'auto'},
        headers={'X-Api-Key': 'ia8jDiTKSkupNnMsxwZ1e6um'},
    )
    # 輸出 去被 照片
    if response.status_code == requests.codes.ok:
        rm_test = testimg.split(".")[-2]+"-nobg.jpg"
        with open(testimg.split(".")[-2]+"-nobg.jpg", 'wb') as out:
            out.write(response.content)
    else:
        print("Error:", response.status_code, response.text)

# =====================   進入clarifai   ==========================

    # 給予金鑰
    app = ClarifaiApp(api_key='2ff7f19d333346cf88f7d6c9b382952c')

    # 選擇顏色model
    model = app.models.get('color')

    # 輸入圖片位置+檔名
    image = ClImage(filename= rm_test)

    color_list = model.predict([image])["outputs"][0]["data"]["colors"]
    

    major_color_list = []
    major_id_list = []
    major_pa_list = []
    second_color_list = []
    second_id_list = []
    second_pa_list = []

    color_dic = {
                "color_id": [],
                "color_name": [],
                "color_pa": []
                }

    for color in color_list:
        color_id = color["raw_hex"]
        color_pa = color["value"]
        color_name = color["w3c"]["name"]

        color_dic["color_id"].append(color_id)
        color_dic["color_name"].append(color_name)
        color_dic["color_pa"].append(color_pa)

#        print("color_data= ",  color_name, color_id + "\t", str(color_pa * 100)[:5] + "%")

    
# =================   依照顏色pa數大小重新排序   ======================================
    for i in range(len(color_dic["color_pa"]) - 1):
        for j in range(len(color_dic["color_pa"])-i-1):
            if color_dic["color_pa"][j] < color_dic["color_pa"][j+1]:
                color_dic["color_pa"][j], color_dic["color_pa"][j+1] = color_dic["color_pa"][j+1], color_dic["color_pa"][j]
                color_dic["color_name"][j], color_dic["color_name"][j+1] = color_dic["color_name"][j+1], color_dic["color_name"][j]
                color_dic["color_id"][j], color_dic["color_id"][j+1] = color_dic["color_id"][j+1], color_dic["color_id"][j]


# ==================   將主要色 append 到 major_color_list   ===========================

    major_color_list.append(color_dic["color_name"][0])
    major_id_list.append(color_dic["color_id"][0])
    major_pa_list.append(color_dic["color_pa"][0])
# =====================================================================================

# =================   將次要色 append 到 second_color_list  ===========================
    try:
        second_color_list.append(color_dic["color_name"][1])
        second_id_list.append(color_dic["color_id"][1])
        second_pa_list.append(color_dic["color_pa"][1])

# ================   若沒有第2種顏色 塞入主要顏色資料   =============================
    except:
        second_color_list.append(color_dic["color_name"][0])
        second_id_list.append(color_dic["color_id"][0])
        second_pa_list.append(color_dic["color_pa"][0])

# ====================================================================================

# =================   將其他色 append 到 other_color_list  ===========================
#    try:
#        other_color_list.append(color_dic["color_name"][2])
#        other_id_list.append(color_dic["color_id"][2])
#        other_pa_list.append(color_dic["color_pa"][2])
#    except:
#        other_color_list.append(color_dic["color_name"][0])
#        other_id_list.append(color_dic["color_id"][0])
#        other_pa_list.append(color_dic["color_pa"][0])

# ============   將要的顏色資訊取出來   ================
    mcol_name = major_color_list[0]
    mcol_id = major_id_list[0]
    mcol_pa = major_pa_list[0]

    scol_name = second_color_list[0]
    scol_id = second_id_list[0]
    scol_pa = second_pa_list[0]


# ================   對次要顏色取hsv   =====================
    # 轉10進制
    scol_rgb = tuple(int(scol_id[i:i+2], 16) for i in (1, 3, 5))
    scol_r = scol_rgb[0]
    scol_g = scol_rgb[1]
    scol_b = scol_rgb[2]
    
# =====================================================================
# 定義 rgb 轉 hsv 的 function

    def rgb_to_hsv(r, g, b):
        r, g, b = r/255.0, g/255.0, b/255.0
        mx = max(r, g, b)
        mn = min(r, g, b)
        df = mx-mn
        if mx == mn:
            h = 0
        elif mx == r:
            h = (60 * ((g-b)/df) + 360) % 360
        elif mx == g:
            h = (60 * ((b-r)/df) + 120) % 360
        elif mx == b:
            h = (60 * ((r-g)/df) + 240) % 360
        if mx == 0:
            s = 0
        else:
            s = (df/mx)*100
        v = mx*100
        return h, s, v

# ===================================================
# ===========   將hsv 正規化   ======================
    sh, ss, sv = rgb_to_hsv(scol_r, scol_g, scol_b)
    sh = math.cos(sh*math.pi/180)
    sh = (sh+1)/2
    ss = ss/100
    sv = sv / 100
# ===================================================
# ============   先找次要顏色標籤   ===========

    data = {'MH':[sh], 'MS':[ss], 'MV':[sv]} 
  
    # Create DataFrame 
    df_data = pd.DataFrame(data) 
# ================ !!!!!!!!!!!!!  模型名稱要改(clf)  !!!!!!!!!!!!!!!!!!! ===================
    scol_label = clf.predict(df_data)[0]
    scol_label = str(scol_label)

# ===============   得到 次要顏色標籤後 要將主要顏色資訊轉成 向量   ====================

    mcol_rgb_vec = list(tuple(int(mcol_id[i:i+2], 16) for i in (1, 3, 5)))
    mcol_rgb_vec.append(mcol_pa)
    mcol_rgb_vec = np.array(mcol_rgb_vec)

# ====================================================================================

# ====================   打開MySQL 尋找顏色相似資料   ===============================
    cursor = None
    conn = None

    try:
        # 建立連線 並指定 database
        conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
        cursor = conn.cursor()
    
        # 下 query指令 範例：只找 mcolor_label = 2 的資料
        query = """SELECT pid, pro_name, brand, pic_url, price, mcolor_label, scolor_label, mcolor_id, scolor_id, mcolor_pa, scolor_pa, mcolor_name, scolor_name
                    FROM shoes_data 
                    WHERE scolor_label = %s"""
      
        scolor_label = scol_label

    
        cursor.execute(query, (scolor_label, ))

        emps = cursor.fetchall()
    

        data = {'pid':[], 'pro_name':[], 'brand':[], 'pic_url':[], 'price':[], 'm_label':[], 's_label':[], 'mc_sim':[]} 


    
        for emp in emps:

            # 將次要顏色id 轉成rgb
            rgb_vec = list(tuple(int(emp[7][i:i+2], 16) for i in (1, 3, 5)))

            # append 顏色趴數
            rgb_vec.append(emp[9])
            rgb_vec = np.array(rgb_vec)
            # 歐式距離算顏色相似度
            dist = np.sqrt(np.sum(np.square(rgb_vec - mcol_rgb_vec)))
        
            data['mc_sim'].append(dist)
            data["pid"].append(emp[0])
            data["pro_name"].append(emp[1])
            data["brand"].append(emp[2])
            data["pic_url"].append(emp[3])
            data["price"].append(emp[4])
            data["m_label"].append(emp[5])
            data["s_label"].append(emp[6])
        
    
   
        df = pd.DataFrame(data)

# 關閉連線
    finally:
        cursor.close()

    df1 = df.sort_values(by=['mc_sim'],ascending=[True]).head(100)

    pid_list = list(df1["pid"])
    name_list = list(df1["pro_name"])
    brand_list = list(df1["brand"])
    pic_url_list = list(df1["pic_url"])
    price_list = list(df1["price"])

    recommend_list = []

    for i in range(len(df1["pid"])):
        pro_data = {
            "pid": pid_list[i],
            "pro_name": name_list[i],
            "brand": brand_list[i],
            "pic_url": pic_url_list[i],
            "price": price_list[i]
        }
        recommend_list.append(pro_data)

    return recommend_list



In [30]:
remove_bg("img_1_2_0_shoe.jpg")

[{'pid': 'CI2082-800',
  'pro_name': 'Nike Air Zoom Pegasus 36 (Oklahoma State)',
  'brand': 'nike',
  'pic_url': 'https://c.static-nike.com/a/images/t_PDP_1280_v1/f_auto/b10nnuzkw7iodqzstdct/air-zoom-pegasus-36-texas-mens-running-shoe-r5Rd7m.jpg',
  'price': '130.0'},
 {'pid': '66230_CDB',
  'pro_name': "Men's Moreno - Pence",
  'brand': 'skechers',
  'pic_url': 'https://image.skechers.com/img/productimages/xlarge/66230_CDB_E.jpg',
  'price': '80.0'},
 {'pid': '5526882327952111',
  'pro_name': 'Alperton 2.0 Leather Boots',
  'brand': 'belstaff',
  'pic_url': 'https://cache.mrporter.com/variants/images/5526882327952111/in/w2000_q80.jpg',
  'price': '575'},
 {'pid': '1193A055-600',
  'pro_name': 'GEL-Mai Knit MT',
  'brand': 'asics',
  'pic_url': 'https://asics.scene7.com/is/image/asics/1193A055_600_0020000513_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100',
  'price': '79.99'},
 {'pid': '16635_CHOC',
  'pro_name': "Women's Skechers On the GO Joy - Breezie",
  

In [1]:
recommend_list = [{'pid': 'CI2082-800',
  'pro_name': 'Nike Air Zoom Pegasus 36 (Oklahoma State)',
  'brand': 'nike',
  'pic_url': 'https://c.static-nike.com/a/images/t_PDP_1280_v1/f_auto/b10nnuzkw7iodqzstdct/air-zoom-pegasus-36-texas-mens-running-shoe-r5Rd7m.jpg',
  'price': '130.0'},
 {'pid': '66230_CDB',
  'pro_name': "Men's Moreno - Pence",
  'brand': 'skechers',
  'pic_url': 'https://image.skechers.com/img/productimages/xlarge/66230_CDB_E.jpg',
  'price': '80.0'},
 {'pid': '5526882327952111',
  'pro_name': 'Alperton 2.0 Leather Boots',
  'brand': 'belstaff',
  'pic_url': 'https://cache.mrporter.com/variants/images/5526882327952111/in/w2000_q80.jpg',
  'price': '575'},
 {'pid': '1193A055-600',
  'pro_name': 'GEL-Mai Knit MT',
  'brand': 'asics',
  'pic_url': 'https://asics.scene7.com/is/image/asics/1193A055_600_0020000513_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100',
  'price': '79.99'},
 {'pid': '16635_CHOC',
  'pro_name': "Women's Skechers On the GO Joy - Breezie",
  'brand': 'skechers',
  'pic_url': 'https://image.skechers.com/img/productimages/xlarge/16635_CHOC_E.jpg',
  'price': '65.0'}]

In [2]:
import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

for r in recommend_list:
    pid = r["pid"]
    pro_name = r["pro_name"]
    brand = r["brand"]
    price = r["price"]
    pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
    print(pic_url)

https://d5cb3ed3.ngrok.io/material/product/CI2082-800.jpg
https://d5cb3ed3.ngrok.io/material/product/66230_CDB.jpg
https://d5cb3ed3.ngrok.io/material/product/5526882327952111.jpg
https://d5cb3ed3.ngrok.io/material/product/1193A055-600.jpg
https://d5cb3ed3.ngrok.io/material/product/16635_CHOC.jpg


In [41]:
from linebot import (
    LineBotApi, WebhookHandler
)

import json

# 載入基礎設定檔
secretFileContentJson=json.load(open("../line_secret_key",'r'))
server_url=secretFileContentJson.get("server_url")

# 生成實體物件
line_bot_api = LineBotApi(secretFileContentJson.get("channel_access_token"))


# 找出使用者推薦清單的輪播訊訊息陣列

def Picfindshoe(userid, recommend_list):
    # 取出消息內User的資料
#     user_profile = line_bot_api.get_profile(userid)  
    
    # 準備放入輪播清單的list
    contentsList = []
    for r in recommend_list:
#         print(r["pid"], r["pro_name"], r["brand"], r["price"])
        pid = r["pid"]
        pro_name = r["pro_name"]
        brand = r["brand"]
        price = r["price"]
        pic_url = 'https://'+server_url+'/material/product/'+pid+'.jpg'
#         print(pid, pro_name, brand, pic_url)
        #循環加到 list_col_color

        bubbleDict = {
                "type": "bubble",
                "direction": "ltr",
                "hero": {
                  "type": "image",
                  "url": pic_url,
                  "size": "full",
                  "aspectRatio": "1.51:1",
                  "aspectMode": "fit"
                },
                "body": {
                  "type": "box",
                  "layout": "vertical",
                  "contents": [
                    {
                      "type": "text",
                      "text": brand,
                      "align": "center",
                      "weight": "bold"
                    },
                    {
                      "type": "text",
                      "text": pro_name[:40],
                      "align": "center"
                    },
                    {
                      "type": "text",
                      "text": price,
                      "align": "center"
                    },
                    {
                      "type": "box",
                      "layout": "horizontal",
                      "contents": [
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "加入購物車",
                            "text": "我想買"+pro_name[:10],
                            "data": "tag=cart&product="+pid+"&userid="+str(userid)
                          }
                        },
                        {
                          "type": "button",
                          "action": {
                            "type": "postback",
                            "label": "加入收藏",
                            "text": pro_name[:10]+"加入收藏",
                            "data": "tag=collection&product="+pid+"&userid="+str(userid)
                          }
                        }
                      ]
                    },
                    {
                      "type": "separator"
                    }
                  ]
                },
                "footer": {
                  "type": "box",
                  "layout": "horizontal",
                  "contents": [
                    {
                      "type": "text",
                      "text": "評分",
                      "gravity": "center"
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "1",
                        "data": "rating=1&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "2",
                        "data": "rating=2&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "3",
                        "data": "rating=3&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "4",
                        "data": "rating=4&product="+pid+"&userid="+str(userid)
                      }
                    },
                    {
                      "type": "button",
                      "action": {
                        "type": "postback",
                        "label": "5",
                        "data": "rating=5&product="+pid+"&userid="+str(userid)
                      }
                    }
                  ]
                }
              }

        contentsList.append(bubbleDict)
    Flex_Message = [
        {
          "type": "text",
          "text": "您好\n以下是根據您的上傳的照片，有以下相似的商品"+u"\U0001f600"+"\n"
        },
        {
          "type": "flex",
          "altText": "圖片找鞋推薦",
          "contents": {
            "type": "carousel",
            "contents": contentsList
          }
        }]
#     result_message_array = detect_json_array_to_linebot_message_array(Flex_Message)
    return Flex_Message

Picfindshoe(611, recommend_list)

[{'type': 'text', 'text': '您好\n以下是根據您的上傳的照片，有以下相似的商品😀\n'},
 {'type': 'flex',
  'altText': '圖片找鞋推薦',
  'contents': {'type': 'carousel',
   'contents': [{'type': 'bubble',
     'direction': 'ltr',
     'hero': {'type': 'image',
      'url': 'https://8662b6fd.ngrok.io/material/product/CI2082-800.jpg',
      'size': 'full',
      'aspectRatio': '1.51:1',
      'aspectMode': 'fit'},
     'body': {'type': 'box',
      'layout': 'vertical',
      'contents': [{'type': 'text',
        'text': 'nike',
        'align': 'center',
        'weight': 'bold'},
       {'type': 'text',
        'text': 'Nike Air Zoom Pegasus 36 (Oklahoma State',
        'align': 'center'},
       {'type': 'text', 'text': '130.0', 'align': 'center'},
       {'type': 'box',
        'layout': 'horizontal',
        'contents': [{'type': 'button',
          'action': {'type': 'postback',
           'label': '加入購物車',
           'text': '我想買Nike Air Z',
           'data': 'tag=cart&product=CI2082-800&userid=611'}},
         {'

In [29]:
cursor = None
conn = None

data = {'MH':[0.1], 'MS':[0.5], 'MV':[0.8]} 

# Create DataFrame 
df_data = pd.DataFrame(data) 
scol_label = clf.predict(df_data)[0]
scol_label = str(scol_label)


try:
    # 建立連線 並指定 database
    conn = mysql.connector.connect(user='root', password='123456', host = 'chatbot_db', database='TEST1')
    cursor = conn.cursor()

    # 下 query指令 範例：只找 mcolor_label = 2 的資料
    query = """SELECT pid, pro_name, brand, pic_url, price, mcolor_label, scolor_label, mcolor_id, scolor_id, mcolor_pa, scolor_pa, mcolor_name, scolor_name
                FROM shoes_data 
                WHERE scolor_label = %s"""
    scolor_label = scol_label


    cursor.execute(query, (scolor_label, ))

    emps = cursor.fetchall()
    print(emps)
# 關閉連線
finally:
    cursor.close()

[('1011A002-400', 'DynaFlyte 3', 'asics', 'https://asics.scene7.com/is/image/asics/1011A002_400_0020000231_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100', '99.95', 2, 3, '#0c75ac', '#1b243d', 0.55, 0.19, 'SteelBlue', 'Black'), ('1011A002-401', 'DynaFlyte 3', 'asics', 'https://asics.scene7.com/is/image/asics/1011A002_401_0020000231_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100', '99.95', 2, 3, '#103b49', '#397a8b', 0.344, 0.283, 'DarkSlateGray', 'SlateGray'), ('1011A008-400', 'GEL-Cumulus 20', 'asics', 'https://asics.scene7.com/is/image/asics/1011A008_400_SR_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100', '69.95', 2, 3, '#0e76ae', '#13192c', 0.29, 0.22, 'SteelBlue', 'Black'), ('1011A019-300', 'GEL-Kayano 25', 'asics', 'https://asics.scene7.com/is/image/asics/1011A019_300_0020000260_RT?$dw_temp_default_gallery$&wid=1280&hei=640&extend=10,10,10,10&qlt=100', '99.95', 2, 3, '#0897ae', '#62c460', 0.643, 0.127, '